# Keras Hyperparametr Tuning with ScikitLearn GridSearchCV

Use [SciKeras.](https://www.adriangb.com/scikeras/stable/index.html) for hyper parameter tuning using Scikit Learn.

* [SciKeras.](https://www.adriangb.com/scikeras/stable/index.html)

> The goal of scikeras is to make it possible to use Keras/TensorFlow with sklearn. This is achieved by providing a wrapper around Keras that has an Scikit-Learn interface. SciKeras is the successor to tf.keras.wrappers.scikit_learn, and offers many improvements over the TensorFlow version of the wrappers. See Migration for a more details.

* [How to Grid Search Hyperparameters for Deep Learning Models in Python with Keras](https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/)

> Keras models can be used in scikit-learn by wrapping them with the KerasClassifier or KerasRegressor class from the module SciKeras. 

* [How to pass callbacks to scikit_learn wrappers (e.g. KerasClassifier)](https://github.com/keras-team/keras/issues/4278)

> ```
> model = KerasClassifier(build_fn=create_model, epochs=100, verbose=1, validation_split=.2)
> grid_search = GridSearchCV(model, para_grid, n_jobs=-1, cv=5, refit='accuracy')
> grid_search.fit(X, y, callbacks=EarlyStopping(monitor='val_loss', patience=3))
> ```

## KerasClassifier

```KerasClassifier``` has its required parameters so that it can compile keras Model.

* [scikeras.wrappers.KerasClassifier](https://www.adriangb.com/scikeras/stable/generated/scikeras.wrappers.KerasClassifier.html#scikeras-wrappers-kerasclassifier)

### Prefix

Prefix is to identify the target instances.

* [Special prefixes](https://www.adriangb.com/scikeras/stable/notebooks/Basic_Usage.html#7.1-Special-prefixes)

> * ```model__``` prefix to specify the parameters for build_fn function
> * ```optimizer__``` prefix to specify the parmeters for the optimizer instance arg the model.compile().

See [optimizers](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) for the optimizers and their parameters.


---
# Setup

In [1]:
import os
import sys
import pathlib
import numpy as np
import tensorflow as tf
from tensorflow import keras   # MUST to make sure of using tensorflow.keras, not keras
from tensorflow.keras.models import (
    Model,
    Sequential
)
from tensorflow.keras.layers import (
    Layer,
    Dense, 
    Dropout, 
    Flatten, 
    Normalization,
    BatchNormalization,
    Activation,
    Conv2D, 
    MaxPooling2D,
)

import sklearn
from sklearn.model_selection import (
    GridSearchCV
)
from scikeras.wrappers import (
    KerasClassifier, 
)

In [2]:
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 1.1.3.


## PYTHONPATH

In [3]:
path_to_lib: str = str(pathlib.Path(os.path.join(os.getcwd(), "../../../lib")).resolve())
assert isinstance(path_to_lib, str)
sys.path.append(path_to_lib)

In [4]:
%load_ext autoreload
%autoreload 2

from util_datetime import (
    get_datetime_string,
)
from util_file import (
    rmdir
)
from util_matplotlib.image import (
    plot_images,
)
from util_tf.nn import (
    LAYER_NAME_NORM,
    LAYER_NAME_CONV2D,
    LAYER_NAME_ACTIVATION,
    LAYER_NAME_MAXPOOL2D,
    LAYER_NAME_DENSE,
    LAYER_NAME_FLAT,
    LAYER_NAME_BN,
    CUSTOM_LAYER_NAME_CONV2DBLOCK,
    build_layers,
    build_nn_model,
    train,
    get_early_stopping_callback,
    get_tensorboard_callback,
    get_confusion_matrix,
    get_precision,
    get_recall,
)

# CIFAR-10

In [5]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
input_shape = x_train[0].shape
number_of_classes = 10

In [6]:
max_value = float(np.max(x_train))
x_train_normed, x_test_normed = x_train/max_value, x_test/max_value

# Model

In [7]:
layers_config = {
    "conv01": {
        "kind": LAYER_NAME_CONV2D, "kernel_size":(3,3), "filters":32, "strides":1, "padding": "same"
    },  
    "act01": {
        "kind": LAYER_NAME_ACTIVATION,
        "activation": "leaky_relu",
        "slope": 0.1
    },
    "bn": {
        "kind": LAYER_NAME_BN
    },
    "maxpool01": {
        "kind": LAYER_NAME_MAXPOOL2D, "pool_size":(2,2), "strides":(2,2), "padding": "valid"
    },
    "flat": {
        "kind": LAYER_NAME_FLAT, "data_format": "channels_last"
    },
    "full01": {
        "kind": LAYER_NAME_DENSE, "units": 100, "activation": "relu", "l2": 1e-2
    },
    "label": {
        "kind": LAYER_NAME_DENSE, "units": number_of_classes, "activation": "softmax"
    }
}

In [8]:
model = KerasClassifier(
    model=build_nn_model,
    # build_nn_model parameters
    model_name="cifar10",
    input_shape=input_shape,
    layers_config=layers_config,
    
    # Keras Model fit() parameters
    verbose=2
)

## Early stop callback

In [9]:
earlystop_callback = get_early_stopping_callback(
    monitor='val_loss', 
    patience=5,
    verbose=1,
    restore_best_weights=True,
)

# Grid Search

In [10]:
param_grid = {
    'epochs': [10, 20],
    'batch_size': [64, 128],
    "optimizer": ["adam", "sgd"],
    'optimizer__learning_rate': [8e-4, 1e-3, 2e-3],
    'model__metrics': [['accuracy'], ['auc']]
}

In [11]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=5)
grid_result = grid.fit(
    X=x_train, 
    y=y_train,
    callbacks=[
        earlystop_callback
    ]
)

2023-02-18 00:08:50.097609: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/10


2023-02-18 00:08:50.326739: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/10


2023-02-18 00:08:50.710667: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-02-18 00:08:50.766666: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-02-18 00:08:50.812486: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-02-18 00:08:50.823056: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-02-18 00:08:50.830840: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-02-18 00:08:50.844571: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


...

313/313 - 31s - loss: 1.6796 - accuracy: 0.4800 - 31s/epoch - 100ms/step
Epoch 19/20
313/313 - 31s - loss: 1.6832 - accuracy: 0.4669 - 31s/epoch - 100ms/step
Epoch 19/20
313/313 - 32s - loss: 1.6944 - accuracy: 0.4538 - 32s/epoch - 102ms/step
Epoch 19/20
313/313 - 31s - loss: 1.6907 - accuracy: 0.4705 - 31s/epoch - 99ms/step
Epoch 20/20
313/313 - 31s - loss: 1.6676 - accuracy: 0.4625 - 31s/epoch - 98ms/step
Epoch 20/20
313/313 - 31s - loss: 1.6901 - accuracy: 0.4743 - 31s/epoch - 99ms/step
Epoch 20/20
313/313 - 31s - loss: 1.6817 - accuracy: 0.4794 - 31s/epoch - 99ms/step
Epoch 20/20
313/313 - 31s - loss: 1.6757 - accuracy: 0.4696 - 31s/epoch - 100ms/step
Epoch 20/20
313/313 - 31s - loss: 1.6803 - accuracy: 0.4588 - 31s/epoch - 100ms/step
Epoch 20/20
313/313 - 31s - loss: 1.6867 - accuracy: 0.4737 - 31s/epoch - 99ms/step
313/313 - 31s - loss: 1.6743 - accuracy: 0.4625 - 31s/epoch - 99ms/step
79/79 - 2s - 2s/epoch - 31ms/step
79/79 - 2s - 2s/epoch - 28ms/step
313/313 - 30s - loss: 1.691

/Users/oonisim/venv/ml/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
120 fits failed out of a total of 240.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/oonisim/venv/ml/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/oonisim/venv/ml/lib/python3.9/site-packages/scikeras/wrappers.py", line 1494, in fit
    super().fit(X=X, y=y, sample_weight=sample_weight, **kwargs)
  File "/Users/oonisim/venv/ml/lib/python3.9/site-packages/scikeras/wrappers.py", line 762, in fit
    self._fit(
  File "/User

Epoch 1/20


2023-02-18 02:54:35.507331: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


391/391 - 8s - loss: 3.7578 - accuracy: 0.2591 - 8s/epoch - 20ms/step
Epoch 2/20
391/391 - 7s - loss: 2.0715 - accuracy: 0.3339 - 7s/epoch - 19ms/step
Epoch 3/20
391/391 - 8s - loss: 1.9336 - accuracy: 0.3735 - 8s/epoch - 20ms/step
Epoch 4/20
391/391 - 8s - loss: 1.8574 - accuracy: 0.3998 - 8s/epoch - 20ms/step
Epoch 5/20
391/391 - 8s - loss: 1.8052 - accuracy: 0.4129 - 8s/epoch - 20ms/step
Epoch 6/20
391/391 - 8s - loss: 1.7752 - accuracy: 0.4263 - 8s/epoch - 19ms/step
Epoch 7/20
391/391 - 8s - loss: 1.7487 - accuracy: 0.4339 - 8s/epoch - 19ms/step
Epoch 8/20
391/391 - 8s - loss: 1.7335 - accuracy: 0.4377 - 8s/epoch - 19ms/step
Epoch 9/20
391/391 - 8s - loss: 1.7211 - accuracy: 0.4445 - 8s/epoch - 19ms/step
Epoch 10/20
391/391 - 8s - loss: 1.7110 - accuracy: 0.4462 - 8s/epoch - 19ms/step
Epoch 11/20
391/391 - 8s - loss: 1.7067 - accuracy: 0.4476 - 8s/epoch - 20ms/step
Epoch 12/20
391/391 - 8s - loss: 1.7059 - accuracy: 0.4499 - 8s/epoch - 20ms/step
Epoch 13/20
391/391 - 8s - loss: 1.6

# Result

In [12]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.470300 using {'batch_size': 128, 'epochs': 20, 'model__metrics': ['accuracy'], 'optimizer': 'adam', 'optimizer__learning_rate': 0.001}
0.310480 (0.062190) with: {'batch_size': 64, 'epochs': 10, 'model__metrics': ['accuracy'], 'optimizer': 'adam', 'optimizer__learning_rate': 0.0008}
0.356860 (0.062292) with: {'batch_size': 64, 'epochs': 10, 'model__metrics': ['accuracy'], 'optimizer': 'adam', 'optimizer__learning_rate': 0.001}
0.374180 (0.032393) with: {'batch_size': 64, 'epochs': 10, 'model__metrics': ['accuracy'], 'optimizer': 'adam', 'optimizer__learning_rate': 0.002}
0.390020 (0.027193) with: {'batch_size': 64, 'epochs': 10, 'model__metrics': ['accuracy'], 'optimizer': 'sgd', 'optimizer__learning_rate': 0.0008}
0.330360 (0.059034) with: {'batch_size': 64, 'epochs': 10, 'model__metrics': ['accuracy'], 'optimizer': 'sgd', 'optimizer__learning_rate': 0.001}
0.381480 (0.021886) with: {'batch_size': 64, 'epochs': 10, 'model__metrics': ['accuracy'], 'optimizer': 'sgd', 'optimizer_